In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [5]:
data_dir = "../../../DATA_MAY18/spec1d/"
fname = '2-8h30m-270-spec1d.npy'

In [7]:
data = np.load(data_dir + fname) #object, flag (0, 1), wavelength. flags: 0 -> iamge, 1 -> ivar
data.shape

(143, 2, 5632)

In [13]:
print(data[0].shape)
print(data[0][0].shape)
print(len(data))

(2, 5632)
(5632,)
143


In [12]:
collapsedSpectrum = data[:, 0, :]
ivar = data[:, 1, :]

In [ ]:
def window(data):
    

In [19]:
i = 0
store_left = []
store_right = []
while (i < 1000):
    store_left.append(i)
    i = i + 100
    store_right.append(i)
    i = i - 30
    
print(len(store_left))
print(len(store_right))

15
15
